In [625]:
import numpy as np
from pyfinite import ffield

In [626]:
string_to_char = {"0000": 'f', "0001": 'g', "0010": 'h', "0011": 'i', "0100": 'j', "0101": 'k', "0110": 'l', "0111": 'm',
 "1000": 'n', "1001": 'o', "1010": 'p', "1011": 'q', "1100": 'r', "1101": 's', "1110": 't', "1111": 'u'}

char_to_string = {'f' : "0000", 'g' : "0001", 'h' : "0010", 'i' : "0011", 'j' : "0100", 'k' : "0101", 'l' : "0110", 'm' : "0111",
 'n' : "1000", 'o' : "1001", 'p' : "1010", 'q' : "1011", 'r' : "1100", 's' : "1101", 't' : "1110", 'u' : "1111"}

In [627]:
def pair_to_binary(st): #It maps a pair of characters to binary string representation of a byte 
    ret = char_to_string[st[0]]+char_to_string[st[1]]
    return ret

def int_to_pair(b): #It converts byte to corresponding two characters

    byte_string = bin(b)
    byte_string = byte_string[2:]

    lencur = len(byte_string) #need to pad with zeros so that the length of string is 8
    for i in range(8-lencur):
        byte_string='0'+byte_string
    
    # print(byte_string)
    char1 = string_to_char[byte_string[0:4]]
    char2 = string_to_char[byte_string[4:8]]

    return char1+char2


def binary_string_to_int(st):
    return int(st, 2)

def block_to_binary(c):
    ret = ""
    for i in range(0, len(c), 2):
        ret += pair_to_binary(c[i:i+2])
    return ret

In [628]:
print(int_to_pair(104))
print(binary_string_to_int("01001110"))
print(pair_to_binary("jt"))
block_to_binary("fghjklmnfghjklmn")

ln
78
01001110


'0000000100100100010101100111100000000001001001000101011001111000'

In [629]:
#define some reusable functinos

F = ffield.FField(7) #create the field of size 2^7 (not 2^8)
exp_store = [[0]*128 for i in range(128)]

def Add(a, b):
    return F.Add(a, b) 

def Multiply(a, b):        
    return F.Multiply(a, b)

def preExp():
    for i in range(128):
        exp_store[i][1] = i
        for j in range(2, 128):
            exp_store[i][j] = F.Multiply(i, exp_store[i][j-1])

def Exp(no, pow):
    return exp_store[no][pow]

preExp() # fill Exp table

In [630]:
def Add_vectors(v1, v2): 
    if(len(v1)!=len(v2)):
        return "ERROR! DIMENSIONS OF VECTORS IN ADDITION DON'T MATCH"
    result = [0, 0, 0, 0, 0, 0, 0, 0]

    for i in range(len(v1)):
        result[i] = Add(v1[i], v2[i])
    return result

def Mul_vector_scalar(v, cons): #multiply vector v by cons (scalar multiplication) in the field F
    result = [0, 0, 0, 0, 0, 0, 0, 0]
    
    for i in range(len(v)):
        result[i] = Multiply(v[i], cons)
    return result

def Mul_matrix_vector(M, V):
    res = [0, 0, 0, 0, 0, 0, 0, 0]

    for r, ele in zip(M, V):
        res = Add_vectors(Mul_vector_scalar(r, ele), res)
    return res

In [631]:
print(Add_vectors([2,3, 0, 0, 0 , 0,0 ,0], [3, 4, 0, 0, 0, 0, 0, 0]))
print(Mul_vector_scalar([2,3, 0, 0, 0 , 0,0 ,0], 6))
print(Mul_matrix_vector([[1, 0, 0, 0, 0, 0, 0 ,0], [0, 1, 0, 0, 0, 0, 0 ,0], [0, 0, 1, 0, 0, 0, 0 ,0], [0, 0, 0, 1, 0, 0, 0 ,0], [0, 0, 0, 0, 1, 0, 0 ,0], [0, 0, 0, 0, 0, 1, 0 ,0], [0, 0, 0, 0, 0, 0, 1 ,0], [0, 0, 0, 0, 0, 0, 0 ,1]], [2, 2, 6, 6, 9, 2, 1, 6]))

[1, 7, 0, 0, 0, 0, 0, 0]
[12, 10, 0, 0, 0, 0, 0, 0]
[2, 2, 6, 6, 9, 2, 1, 6]


In [632]:

candidate_A = [[[] for i in range(8)] for j in range(8)]
candidate_exp = [[] for i in range(8)]

inp_file = open("inputs.txt", 'r')
inputs = inp_file.readlines()

out_file = open("outputs.txt", 'r')
outputs = out_file.readlines()

# print(inputs)
# print(outputs)

cur_pos=0 #we look at all bytes of the input set one after the other. cur_pos tells the current byte we are looking at

#look at ith byte of input when cur_pos = i since only cur_pos is varied in that input set

for a in range(len(inputs)): 
    input_set = inputs[a] #input_set is set of inputs in which we vary only the byte at cur_pos, rest of the bytes are "ff"
    output_set = outputs[a]

    cur_byte_for_inputs = []
    cur_byte_for_outputs = []

    for each_input in input_set.strip().split(" "):
        cur_byte_for_inputs.append(binary_string_to_int((block_to_binary(each_input))[8*cur_pos:8*cur_pos+8])) #byte as an int(will be from 0-127)
    for each_output in output_set.strip().split(" "):
        cur_byte_for_outputs.append(binary_string_to_int((block_to_binary(each_output))[8*cur_pos:8*cur_pos+8]))
    
    # print(cur_byte_for_inputs)
    # print(cur_byte_for_outputs)

    for i in range(1, 128): #considering all entries in the A[cur_pos][cur_pos]
        for j in range(1, 127): #exponent of cur_pos 
            
            flag = True
            for lop in range(len(cur_byte_for_inputs)): 
                val_inp = cur_byte_for_inputs[lop]
                cal_out = cur_byte_for_outputs[lop]

                if (cal_out != Exp(Multiply(Exp(Multiply(Exp((val_inp), j), i), j), i), j)): #test if for all inputs, we get the calculated and    given bytes as equal for given i, j. We can test like this since all bytes except cur_pos are "ff", hence value is 0.  
                    
                    flag = False
                    break
            if flag:
                candidate_exp[cur_pos].append(j) #maintain E
                candidate_A[cur_pos][cur_pos].append(i) #maintain A
                # print((i))
                # print((j))
    cur_pos+=1

print("A: ",candidate_A)
print("Expoenents: ", candidate_exp)

A:  [[[27, 84, 84], [], [], [], [], [], [], []], [[], [10, 18, 70], [], [], [], [], [], []], [[], [], [33, 43, 98], [], [], [], [], []], [[], [], [], [12, 38, 113], [], [], [], []], [[], [], [], [], [28, 62, 112], [], [], []], [[], [], [], [], [], [11, 41, 127], [], []], [[], [], [], [], [], [], [27, 71, 92], []], [[], [], [], [], [], [], [], [5, 38, 93]]]
Expoenents:  [[1, 19, 107], [109, 39, 106], [22, 37, 68], [76, 4, 47], [58, 110, 86], [53, 83, 118], [23, 48, 56], [88, 21, 18]]


In [633]:
#got 3 pairs of possible values for values of (A[i][i], E[i]) 
FINAL_A = [[0 for j in range(8)] for i in range(8)]
FINAL_EXP = [0 for i in range(8)]

cur_pos=0
for a in range(len(inputs)): #we aim to finalise the 
    
    if cur_pos==7:
        break

    input_set = inputs[a]
    output_set = outputs[a]
    
    cur_byte_for_inputs = []
    cur_byte_for_outputs = []
    
    for each_input in input_set.strip().split(" "):
        cur_byte_for_inputs.append(binary_string_to_int((block_to_binary(each_input))[8*cur_pos:8*cur_pos+8])) 
    for each_output in output_set.strip().split(" "):
        cur_byte_for_outputs.append(binary_string_to_int((block_to_binary(each_output))[8*cur_pos+8:8*cur_pos+16]))

    # print(cur_byte_for_inputs)
    # print(cur_byte_for_outputs)
    
    for i in range(len(candidate_exp[cur_pos])):
        # print(i)
        expc = candidate_exp[cur_pos][i]
        acc = candidate_A[cur_pos][cur_pos][i]
        # print(expc)
        # print(acc)
        # print("curposss: ", cur_pos)
        # print(candidate_exp[cur_pos])
        # print("lengthh: ",len(candidate_exp[cur_pos+1]))
        for j in range(len(candidate_exp[cur_pos+1])):
            expc1 = candidate_exp[cur_pos+1][j]
            ac1c1 = candidate_A[cur_pos+1][cur_pos+1][j]
            # print("ttff")
            # print(j)
            # print(expc1)
            # print(ac1c1)
            for k in range(128):
                flag=True
                for l in range(len(cur_byte_for_inputs)):
                    inp_val = cur_byte_for_inputs[l]
                    out_val = cur_byte_for_outputs[l]
                    calc_output_byte=Exp(Add(Multiply(Exp(Multiply(Exp(inp_val,expc),acc),expc),k), Multiply(Exp(Multiply(Exp(inp_val,expc),k),expc1),ac1c1)),expc1)
                    if(calc_output_byte!=out_val):
                        flag=False
                        break
                if(flag==True): #we saw that there was only 1 possible value for the diagonal and off-diagonal values of A. Hence, we create a final matrix for A and keep updating it since there is no ambiguity now
                    # candidate_A[cur_pos][cur_pos].append(acc)
                    # candidate_A[cur_pos+1][cur_pos+1].append(ac1c1)
                    # candidate_A[cur_pos+1][cur_pos].append(k)
                    # candidate_exp[cur_pos].append(expc)
                    # candidate_exp[cur_pos+1].append(expc1)
                    FINAL_A[cur_pos][cur_pos]=acc
                    FINAL_A[cur_pos+1][cur_pos+1]=ac1c1
                    FINAL_A[cur_pos][cur_pos+1]=k
                    FINAL_EXP[cur_pos]=expc
                    FINAL_EXP[cur_pos+1]=expc1
                    continue
    cur_pos+=1

print(FINAL_A)
print("\nObtained EXP: ", FINAL_EXP)

[[84, 115, 0, 0, 0, 0, 0, 0], [0, 70, 17, 0, 0, 0, 0, 0], [0, 0, 43, 25, 0, 0, 0, 0], [0, 0, 0, 12, 111, 0, 0, 0], [0, 0, 0, 0, 112, 101, 0, 0], [0, 0, 0, 0, 0, 11, 82, 0], [0, 0, 0, 0, 0, 0, 27, 10], [0, 0, 0, 0, 0, 0, 0, 38]]

Obtained EXP:  [19, 106, 37, 76, 86, 53, 23, 21]


In [634]:
def EAEAE(plain_text, A, E): #We define a functino to simulate the encryption on a plaintext block
    pt = [0 for i in range(8)]
    for ind in range(8):
        pt[ind]=binary_string_to_int(plain_text[8*ind:8*ind+8])
    # print(pt)
    M1 = [0 for i in range(8)]
    M3 = [0 for i in range(8)]
    M5 = [0 for i in range(8)]
    
    for i in range(len(pt)):
        M1[i] = Exp(pt[i], E[i])
    # print(M1)
    # print(A)
    M2 = Mul_matrix_vector(A, M1)
    # print(M2)

    for i in range(len(pt)):
        M3[i] = Exp(M2[i], E[i])

    M4 = Mul_matrix_vector(A, M3)

    for i in range(len(pt)):
        M5[i] = Exp(M4[i], E[i])

    return M5

In [635]:
print(FINAL_A)

print(EAEAE("0000000100000000000000000000000000000000000000000000000000000000", FINAL_A, FINAL_EXP))

[[84, 115, 0, 0, 0, 0, 0, 0], [0, 70, 17, 0, 0, 0, 0, 0], [0, 0, 43, 25, 0, 0, 0, 0], [0, 0, 0, 12, 111, 0, 0, 0], [0, 0, 0, 0, 112, 101, 0, 0], [0, 0, 0, 0, 0, 11, 82, 0], [0, 0, 0, 0, 0, 0, 27, 10], [0, 0, 0, 0, 0, 0, 0, 38]]
[67, 78, 101, 0, 0, 0, 0, 0]


In [636]:

for start in range(6):
    temp_A = FINAL_A

    for i in range(len(inputs)): #try to determine A[start+det][start] for all start, A[start][start] and A[start+1][start] are full
        if(i+start+2>7):
            continue

        input_set = inputs[i]
        output_set = outputs[i]

        # print(input_set)
        # print(output_set)

        all_inputs = []
        all_outputs = []
        
        for each_input in input_set.strip().split(" "):
            all_inputs.append((block_to_binary(each_input)))

        for each_output in output_set.strip().split(" "):
            all_outputs.append((block_to_binary(each_output)))
        
        # print(all_inputs) #binary strings of blocks
        # print(all_outputs)

        c=0
        for k in range(1, 128):
            temp_A[i][i+start+2]=k
            
            # print(len(all_inputs))
            flag=True
            for ll in range(len(all_inputs)):
                inp_val = all_inputs[ll]
                out_val = all_outputs[ll]

                # print("start, i, k, ll:", start, i, k, ll)

                # print(all_inputs[ll])
                # print(all_outputs[ll])
                # print("value at FINAL[i+start+2][i]:", k)
                # print("     output ki ye value: ",(all_outputs[ll][8*(i+start+2):8*(i+start+2)+8]))
                # print("input going into EAEAE:  ", all_inputs[ll])
                # print("calcualted output value: ",EAEAE(all_inputs[ll], FINAL_A, FINAL_EXP))
                if(binary_string_to_int(out_val[8*(i+start+2):8*(i+start+2)+8]) != EAEAE(inp_val, temp_A, FINAL_EXP)[i+start+2]):
                    flag = False
                    # print("above combo didnt match")
                    break
            if(flag==True):
                if(c):
                    print("Ambiguity still exists")
                c+=1
                FINAL_A[i][i+start+2]=k
                
                break 
                    
        if(FINAL_A[i][i+start+2]==127):
            FINAL_A[i][i+start+2]=0
                 

print(FINAL_A)
print(FINAL_EXP)

[[84, 115, 26, 97, 98, 31, 15, 89], [0, 70, 17, 26, 33, 38, 120, 14], [0, 0, 43, 25, 12, 28, 0, 85], [0, 0, 0, 12, 111, 43, 104, 22], [0, 0, 0, 0, 112, 101, 30, 13], [0, 0, 0, 0, 0, 11, 82, 68], [0, 0, 0, 0, 0, 0, 27, 10], [0, 0, 0, 0, 0, 0, 0, 38]]
[19, 106, 37, 76, 86, 53, 23, 21]


In [637]:
#no ambiguity exists. Hence we have determined A
#Two halves the given password gsmgfnjqlgihkoluifkmgoftjfiijgfn
password_half_1 = "gsmgfnjqlgihkolu"
password_half_2 = "ifkmgoftjfiijgfn"

#Go over all byte values(0 to 127) and see if output byte matches calculated byte
def Decrypt(password):
    passw = block_to_binary(password)
    ret = ""
    for ind in range(8):
        for ans in range(128):
            inp = ret + int_to_pair(ans)+'f'*(16-len(ret)-2) #bytes after ans dont effect (ind)th index of output due to lower traingular matrix A
            if binary_string_to_int(passw[8*ind:8*ind+8]) == EAEAE(block_to_binary(inp), FINAL_A, FINAL_EXP)[ind]:
                ret += int_to_pair(ans)
                break
    return ret

fst = Decrypt(password_half_1)
scd = Decrypt(password_half_2)

print(fst)
print(scd)

milqltmhmnmpmfmh
lglrifififififif


In [638]:
def get_ascii_value(a):
    return (chr)((ord(a[0])-ord('f'))*16 + ord(a[1])-ord('f'))

In [639]:
for i in range(0, len(fst), 2):
    print(get_ascii_value(fst[i]+fst[i+1]), end="")

for i in range(0, len(scd), 2):
    print(get_ascii_value(scd[i]+scd[i+1]), end="")

sknrxzpral000000